In [67]:
#script for taking the parameters from blue pebble (BPoptimisedParameters) and sampling the circuits to get results distributions
import sys
import os
import re
import json
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'

# Packages for quantum stuff
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import (
    FakeBrisbane,
    FakeSherbrooke,
    FakeTorino,
)  

In [68]:
#\\\/////////    Functions    //////////
def findFile(directory, identifier):
    identifierLower = identifier.lower()
    try:
        for filename in os.listdir(directory):
            if identifierLower in filename.lower():
                return filename
    except FileNotFoundError:
        print(f"Error: No file found for the folder/identifier combination: {directory} + {identifier}")
        return None
    return None

def load_qubo_and_build_hamiltonian(file_path, instance_id):
    """
    Loads QUBO terms, weights, and constant from a JSON file.
    Determines the number of qubits from the terms and constructs
    the Hamiltonian as a Qiskit SparsePauliOp.
    """

    with open(file_path, "r") as f:
        all_qubos_data = json.load(f)  # Assumes this loads a list of dicts

    selected_qubo_data = None
    # Find the dictionary for the specified instance_id
    for qubo_instance in all_qubos_data:
        if (
            qubo_instance["instance_id"] == instance_id
        ):  # Assumes 'instance_id' exists and is correct
            selected_qubo_data = qubo_instance
            break

    terms = selected_qubo_data["terms"]
    weights = selected_qubo_data["weights"]
    constant = selected_qubo_data.get("constant", 0.0)
    problemType = selected_qubo_data.get("problem_type")

    pauli_list = []
    num_qubits = 0

    if terms:
        # Flatten the list of lists and filter out empty sublists or non-integer elements
        all_indices = []
        for term_group in terms:
            for idx in term_group:
                all_indices.append(idx)
        num_qubits = max(all_indices) + 1

    for term_indices, weight in zip(terms, weights):
        if not term_indices or not all(isinstance(idx, int) for idx in term_indices):
            # Skip if term_indices is empty or contains non-integers
            continue

        paulis_arr = ["I"] * num_qubits
        if len(term_indices) == 1:  # Linear term
            paulis_arr[term_indices[0]] = "Z"
        elif len(term_indices) == 2:  # Quadratic term
            paulis_arr[term_indices[0]] = "Z"
            paulis_arr[term_indices[1]] = "Z"

        pauli_list.append(("".join(paulis_arr)[::-1], weight))

    if (
        not pauli_list and num_qubits > 0
    ):  # No valid Pauli terms were created, but num_qubits > 0
        cost_hamiltonian = SparsePauliOp(
            ["I"] * num_qubits, [0]
        )  # Zero operator on n_qubits
    elif not pauli_list and num_qubits == 0:
        cost_hamiltonian = SparsePauliOp(
            "I", [0]
        )  # Placeholder for 1 qubit if everything is empty
    else:
        cost_hamiltonian = SparsePauliOp.from_list(pauli_list)

    return cost_hamiltonian, constant, num_qubits, problemType

def save_results_to_json(
    resultsDict, problemClass, qubits, layers, backendName
):
    filename = f"finalSamplingResults/{problemClass}_{qubits}q_p={layers}_{backendName}.json"

    with open(filename, "w") as f:
        json.dump(resultsDict, f, indent=4)

def parse_log_to_dict(directory, identifier):
    targetFile = findFile(directory, identifier)

    pattern = re.compile(
        r"Instance: (?P<instance>\d+), "
        r"Time: (?P<time>[\d.]+)s, "
        r"Number of optimisation loops: (?P<loops>\d+), "
        r"Cost: (?P<cost>[\d.]+), "
        r"Params: \[(?P<params>.*?)\]"
    )

    results_dict = {}
    try:
        with open(f"{directory}/{targetFile}", 'r') as f:
            for line in f:
                match = pattern.match(line)
                if match:
                    # Extract captured groups by name
                    data = match.groupdict()

                    # Convert values to the correct data types
                    instance_id = int(data['instance'])
                    
                    # Parse the space-separated numbers inside the 'Params' string
                    param_list = [float(p) for p in data['params'].split()]
                    
                    # Build the nested dictionary for this instance
                    results_dict[instance_id] = {
                        "Time": float(data['time']),
                        "Number of optimisation loops": int(data['loops']),
                        "Cost": float(data['cost']),
                        "Params": param_list
                    }
    except FileNotFoundError:
        print(f"Error: The file '{targetFile}' was not found.")
        return None

    return results_dict

In [69]:
#//////////// Variables //////////
problemType = "TSP"  # "Knapsack" or "TSP"
samplingInstances = 100 # number of circuits that are formed and sampled aka number of distribution generated
depth = 1 # number of layers in the QAOA circuit
#backendSimulator = AerSimulator()
backendSimulator = AerSimulator.from_backend(FakeTorino())

In [70]:
#//////////// Loading Stuff in //////////
quboFileName = f"studyQuboBatches/{findFile('studyQuboBatches', problemType)}"
print(f"Loading QUBO data from: {quboFileName}")
parameters = parse_log_to_dict("BPoptimisedParameters", problemType)
print(parameters)

Loading QUBO data from: studyQuboBatches/batch_QUBO_data_TSP_9q_.json
{2: {'Time': 72.7394, 'Number of optimisation loops': 20, 'Cost': 100.216796875, 'Params': [1.01660252, 1.82755103]}, 8: {'Time': 80.0874, 'Number of optimisation loops': 23, 'Cost': 58.06982421875, 'Params': [0.34177584, -0.02833084]}, 9: {'Time': 81.0744, 'Number of optimisation loops': 21, 'Cost': 141.21875, 'Params': [2.06491691, 1.1534262]}, 10: {'Time': 83.6202, 'Number of optimisation loops': 22, 'Cost': 119.5185546875, 'Params': [1.2574451, 2.19903454]}, 6: {'Time': 83.8963, 'Number of optimisation loops': 24, 'Cost': 54.05078125, 'Params': [3.59931953, 3.11457649]}, 5: {'Time': 87.5609, 'Number of optimisation loops': 23, 'Cost': 110.08837890625, 'Params': [2.86695257, -0.57959944]}, 4: {'Time': 91.3987, 'Number of optimisation loops': 26, 'Cost': 84.21435546875, 'Params': [2.38994012, 1.91033395]}, 1: {'Time': 92.428, 'Number of optimisation loops': 25, 'Cost': 100.78955078125, 'Params': [1.25696832, 0.9399

In [71]:
#///////////// Main Script /////////////
samplingResults = {}
for instanceIndex in range(1, 1 + samplingInstances):
    print(f"Processing instance {instanceIndex}...")
    cost_hamiltonian, constant_offset, num_qubits, problem_type = (
        load_qubo_and_build_hamiltonian(quboFileName, instanceIndex)
    )
    circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=depth)
    circuit.measure_all()
    pm = generate_preset_pass_manager(optimization_level=3, backend=backendSimulator)
    candidate_circuit = pm.run(circuit)
    
    betaGamma = parameters[instanceIndex]["Params"]
    optimized_circuit = candidate_circuit.assign_parameters(betaGamma)
    sampler = Sampler(mode=backendSimulator)
    sampler.options.default_shots = 1000

    result = sampler.run([optimized_circuit]).result()
    dist = result[0].data.meas.get_counts()
    samplingResults[instanceIndex] = dist

save_results_to_json(
    samplingResults, problemType, num_qubits, depth, backendSimulator.name
)


Processing instance 1...
Processing instance 2...
Processing instance 3...
Processing instance 4...
Processing instance 5...
Processing instance 6...
Processing instance 7...
Processing instance 8...
Processing instance 9...
Processing instance 10...
Processing instance 11...
Processing instance 12...
Processing instance 13...
Processing instance 14...
Processing instance 15...
Processing instance 16...
Processing instance 17...
Processing instance 18...
Processing instance 19...
Processing instance 20...
Processing instance 21...
Processing instance 22...
Processing instance 23...
Processing instance 24...
Processing instance 25...
Processing instance 26...
Processing instance 27...
Processing instance 28...
Processing instance 29...
Processing instance 30...
Processing instance 31...
Processing instance 32...
Processing instance 33...
Processing instance 34...
Processing instance 35...
Processing instance 36...
Processing instance 37...
Processing instance 38...
Processing instance 3